# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation. To run this example Julia's `PyCall` package
needs to be able to find an installation of `pymatgen`.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single k-point and a small `Ecut` of 5 Hartree.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=(1, 1, 1))

PlaneWaveBasis discretization:
    Ecut                 : 5.0 Ha
    fft_size             : (32, 32, 32), 32768 total points
    kgrid type           : Monkhorst-Pack
    kgrid                : [1, 1, 1]
    num. irred. kpoints  : 1

    Discretized Model(lda_xc_teter93, 3D):
        lattice (in Bohr)    : [0         , 10.2631   , 10.2631   ]
                               [10.2631   , 0         , 10.2631   ]
                               [10.2631   , 10.2631   , 0         ]
        unit cell volume     : 2162.1 Bohr³
    
        atoms                : Si₁₆
        atom potentials      : ElementPsp(Si, psp="hgh/lda/si-q4")
    
        num. electrons       : 64
        spin polarization    : none
        temperature          : 0 Ha
    
        terms                : Kinetic()
                               AtomicLocal()
                               AtomicNonlocal()
                               Ewald()
                               PspCorrection()
                               

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.116616e+02     1.580269e+00
 * time: 0.4233400821685791
     1     1.005937e+01     9.025184e-01
 * time: 1.5696320533752441
     2    -1.248171e+01     1.051114e+00
 * time: 1.6400749683380127
     3    -3.446720e+01     8.113705e-01
 * time: 1.7360429763793945
     4    -4.791723e+01     5.527958e-01
 * time: 1.8456201553344727
     5    -5.714824e+01     2.684216e-01
 * time: 1.9288229942321777
     6    -5.979659e+01     2.012331e-01
 * time: 1.9916551113128662
     7    -6.087494e+01     8.307528e-02
 * time: 2.0543270111083984
     8    -6.134750e+01     3.464796e-02
 * time: 2.136775016784668
     9    -6.162156e+01     2.920304e-02
 * time: 2.1996450424194336
    10    -6.179877e+01     3.146358e-02
 * time: 2.262197971343994
    11    -6.191388e+01     2.515421e-02
 * time: 2.3247599601745605
    12    -6.200980e+01     2.144786e-02
 * time: 2.387629985809326
    13    -6.208807e+01     1.964651e-02
 * time: 2.4623420238494

In [5]:
scfres.energies

Energy breakdown (in Ha):
    Kinetic             25.7671084
    AtomicLocal         -18.8557706
    AtomicNonlocal      14.8522649
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485387 
    Xc                  -19.3336827

    total               -62.261666457410